In [3]:
#pip install torchaudio

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.9 MB/s eta 0:00:02
   ----------------- ---------------------- 1.0/2.5 MB 1.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.6/2.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------- ----- 2.1/2.5 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/204.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/204.2 MB 882.6 kB/s eta 0:03:51
   ---------------------------------------- 0.8/204.2 MB 1.1 MB/s eta 0:03:02
   ---------------------------------------- 1.3/204.2 MB 1.4 MB/s eta 0:02:23
   ---------------------------------------- 1.6/204.2 MB 1.4 MB/s eta 0:02:21
   -----------------------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#pip install --upgrade torchaudio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import torchaudio
from torchaudio.transforms import Resample, MelSpectrogram, InverseMelScale, GriffinLim
import os
from IPython.display import Audio

c:\Users\jpeir\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
def log_scale_spectrogram(spectrogram, epsilon=0.001):
    # Aplicar logaritmo con epsilon
    log_spec = torch.log(spectrogram + epsilon)

    # Normalizar al rango [0,1]
    log_spec_min = log_spec.min()
    log_spec_max = log_spec.max()
    log_spec_norm = (log_spec - log_spec_min) / (log_spec_max - log_spec_min)
    
    return log_spec_norm, log_spec_max, log_spec_min

In [12]:
def Reconstruction(log_spectr, maxi, mini,
                   sample_rate = 16000,
                   n_mels = 256, n_fft=1024,
                   hop_length=260, epsilon=0.001,
                   num_iters=500):
    
    # Desnormalizar
    log_spec = log_spectr * (maxi - mini) + mini

    # Deslogarizar
    mel = torch.exp(log_spec) - epsilon

    # Desmelizar
    mel_to_stft = InverseMelScale(n_stft = n_fft // 2 + 1,
                                  n_mels=n_mels,
                                  sample_rate=sample_rate,
                                  driver='gelsd')
    spectrogram = mel_to_stft(mel)

    # Desespectogramizar
    griffin_lim = GriffinLim(n_fft=n_fft, hop_length=hop_length, power=1.0, n_iter=num_iters)
    waveform = griffin_lim(spectrogram)

    return waveform

In [ ]:
audio_path = "./PQMD/00a6f79ade0f483a963452eb9245df2d.wav"

# Cargar el audio
waveform, samp_rt = torchaudio.load(audio_path)

waveform = waveform.mean(axis=0)

sample_rate = 16000
resampler = Resample(orig_freq=samp_rt, new_freq=sample_rate)
waveform = resampler(waveform)

mel_transform = MelSpectrogram(
    sample_rate=sample_rate,   # Tasa de muestreo del audio
    n_fft=1024,                # Tamaño de la ventana FFT
    win_length=1024,
    hop_length=260,            # Hop length (desplazamiento entre ventanas)
    n_mels=256,                # Número de bandas Mel
    power=1.0,                  # Magnitud (espectrograma de potencia)
    f_min=0,
    f_max=16000 / 2,
    center=False,
    window_fn=torch.hann_window,
    mel_scale='htk',
    norm=None
)
spectrogram = mel_transform(waveform)
log_spectr, maxi, mini = log_scale_spectrogram(spectrogram)

print(f"Forma del waveform: {waveform.shape}")
print(f"Tasa de muestreo: {sample_rate} Hz")
print(f"Duration: {waveform.shape[0] / 16000} s")
print(f"Tamaño mel: {log_spectr.shape}")

reconstruction = Reconstruction(log_spectr, maxi, mini)

Forma del waveform: torch.Size([67328])
Tasa de muestreo: 16000 Hz
Duration: 4.208 s
Tamaño mel: torch.Size([256, 256])


In [ ]:
for audio in os.listdir("./PQMD"):
    waveform, sample_rate = torchaudio.load(audio_path)

    waveform = waveform.mean(dim=0, keepdim=True)
    waveform = resampler(waveform)
    spectrogram = mel_transform(waveform)
    log_spectr, maxi, mini = log_scale_spectrogram(spectrogram)

In [14]:
Audio(waveform.numpy(), rate=sample_rate)

In [15]:
Audio(reconstruction.numpy(), rate=sample_rate)